# EDA for Train.csv

# 1. 작업 환경 설정 및 데이터 구조 확인

In [1]:
!pip install hvplot

You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import hvplot
import hvplot.pandas

In [3]:
# data load
train = pd.read_csv('../input/train.csv')

In [4]:
# data 정보 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201917 entries, 0 to 201916
Data columns (total 6 columns):
first_active_month    201917 non-null object
card_id               201917 non-null object
feature_1             201917 non-null int64
feature_2             201917 non-null int64
feature_3             201917 non-null int64
target                201917 non-null float64
dtypes: float64(1), int64(3), object(2)
memory usage: 9.2+ MB


In [5]:
# 결측치 개수 확인
train.isnull().sum()

first_active_month    0
card_id               0
feature_1             0
feature_2             0
feature_3             0
target                0
dtype: int64

## 2. first_active_month
- `first_active_month`는 '처음으로 구매가 발생한 달'을 의미한다.

### 2.1 시간에 따른 first_active_month의 분포

In [6]:
# 각 first_active_month의 개수 확인
first_active_month_count = pd.DataFrame(train['first_active_month'].value_counts()).reset_index()
first_active_month_count.rename(columns = {"first_active_month":'count', 'index':'first_active_month'}, inplace=True)

# first_active_month를 기준으로 오름차순으로 정렬 후 시각화
sorted_first_active_month_count = first_active_month_count.sort_values('first_active_month')
(sorted_first_active_month_count.hvplot.line(x='first_active_month', y='count', rot=90, width=800, title="시간에 따른 first_active_month의 분포") +
sorted_first_active_month_count.hvplot.table(width=300))

:Layout
   .Curve.I :Curve   [first_active_month]   (count)
   .Table.I :Table   [first_active_month,count]

- 전체적으로 보았을 때 지속적으로 증가하다 2017년 9월 정점을 찍은 후 급격히 감소했다.
- 특정 시점에서 갑작스럽게 증가하거나 감소하는 등 급격한 변화가 있었다.
- first_active_month의 증가는 완벽하게 일치하진 않지만, 사용자의 수가 그만큼 증가했다고도 해석할 수 있다. 따라서 first_active_month의 count의 증가는 Elo의 사용자 수가 그만큼 증가했다고도 해석할 수 있다. 또한, 2018년 1월, 2월에 급격하게 하락한 이유 또한 Elo의 사용자 수와 연관 있을 것이라 추측할 수 있다.

In [7]:
# Q. 직전달과 비교했을 때 변화율이 가장 높은 구간은?
sorted_first_active_month_count['pct_change'] = sorted_first_active_month_count['count'].pct_change()
sorted_first_active_month_count.sort_values('pct_change').head()

,first_active_month,count,pct_change
62,2018-01,34,-0.996657
74,2018-02,1,-0.970588
73,2011-12,2,-0.750000
68,2012-12,18,-0.625000
67,2012-06,19,-0.321429


- 직전달과 비교했을 때 2018년 1월과 2월의 변화율이 각각 -99%, -98%로 가장 컸다.
- 이후의 EDA 과정에서 왜 이런 결과가 발생했는지 확인해 볼 필요성이 있어 보인다.

### 2.2 시간에 따른 충성도의 변화

In [8]:
grouped_first_active_month = train.groupby('first_active_month').agg({'target':'mean'}).sort_values('first_active_month').reset_index()
(grouped_first_active_month.hvplot.line(x='first_active_month', y='target', rot=90, title="시간에 따른 충성도의 변화") +
grouped_first_active_month.hvplot.table(width=300))

:Layout
   .Curve.I :Curve   [first_active_month]   (target)
   .Table.I :Table   [first_active_month,target]

### 2.3 `target` 변수의 결측치
- `target` 변수와 다른 변수와의 관계를 보던 중 특이한 값(-33.21928095)을 발견했다.
- 결측치로 의심이 되어 해당 값을 결측치로 볼 수 있는지 확인해보기로 했다.

#### 2.3.1 `feature_1, 2, 3` 세 변수의 모든 카테고리 레벨에서 해당 값이 등장했다.

In [9]:
(train.sort_values('feature_1').hvplot.box('target', by='feature_1' ,subplots=True, shared_axes=False, height=250, width=500) +
train.sort_values('feature_2').hvplot.box('target', by='feature_2' ,subplots=True, shared_axes=False, height=250, width=500) +
train.sort_values('feature_3').hvplot.box('target', by='feature_3' ,subplots=True, shared_axes=False, height=250, width=500)).cols(2)

:Layout
   .BoxWhisker.I   :BoxWhisker   [feature_1]   (target)
   .BoxWhisker.II  :BoxWhisker   [feature_2]   (target)
   .BoxWhisker.III :BoxWhisker   [feature_3]   (target)

- boxplot을 그려본 결과, `feature_1`, `feature_2`, `feature_3`의 카테고리 레벨별 충성도의 평균은 모두 0과 근접했다.
- 또한, 소수점 8자리까지 정확히 일치하는 특정 값(-33.21928095)이 세 변수, 모든 카테고리 레벨에서 등장했다.

In [10]:
train['target'].min()

-33.21928095

- pandas dataframe에선 기본값으로 소수점 표시 제한이 있어서 정확한 수치가 표시되지 않는다.
- 해당 값의 정확한 값은 -33.21928095이다.

#### 2.3.2 해당값이 지나치게 많다.

In [11]:
print(train['target'].value_counts().head())

print('\n해당 row의 개수 : {:,}개'.format(len(train[train['target'] == -33.21928095])))
print('전체 row에서 해당 값의 비율 : {}%'.format(round((len(train[train['target'] == -33.21928095]) / len(train)) * 100, 2)))

-33.219281    2207
 0.000000     1630
 1.000000      117
-1.000000       81
-0.584963       35
Name: target, dtype: int64

해당 row의 개수 : 2,207개
전체 row에서 해당 값의 비율 : 1.09%


- 해당 값은 `target` 값 별 개수는 2,207개로, 가장 많은 부분(1%)을 차지한다.
- 충성도가 0인 고객들을 제외하고 본다면, 비정상적으로 등장 빈도가 높은 값이다.

In [12]:
train[train['target'] > -33.21928095].sort_values('target').head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
46653,2017-02,C_ID_8d4331102a,3,2,1,-17.608147
5888,2017-06,C_ID_3f7ba15ec2,1,1,0,-17.492640
161122,2017-03,C_ID_9636eefa07,1,1,0,-17.441923
8386,2016-11,C_ID_e954ad8a0a,1,1,0,-17.189636
169498,2017-07,C_ID_b889d602a9,2,1,0,-15.835656


- `target` 변수를 오름차순으로 정렬하여 해당 값(-33.21928095)보다 큰 바로 위의 수를 확인했다.
- -17.608147으로 해당 값과는 큰 차이를 보였다.

#### 2.3.3 해당값을 제외한 다른 값들의 범위와 지나치게 차이가 난다.

In [13]:
print(train.sort_values('target', ascending=False).head())
print(train[train['target'] > -33.21928095].sort_values('target').head())

       first_active_month          card_id    ...      feature_3     target
41325             2017-05  C_ID_a4e600deef    ...              1  17.965068
4460              2017-01  C_ID_1c8a5b9d44    ...              1  17.675716
162468            2017-02  C_ID_b0f1d28bd3    ...              1  15.994455
4459              2017-05  C_ID_700c15a07d    ...              1  15.606405
82971             2016-11  C_ID_ecc4e2e188    ...              1  15.294621

[5 rows x 6 columns]
       first_active_month          card_id    ...      feature_3     target
46653             2017-02  C_ID_8d4331102a    ...              1 -17.608147
5888              2017-06  C_ID_3f7ba15ec2    ...              0 -17.492640
161122            2017-03  C_ID_9636eefa07    ...              0 -17.441923
8386              2016-11  C_ID_e954ad8a0a    ...              0 -17.189636
169498            2017-07  C_ID_b889d602a9    ...              0 -15.835656

[5 rows x 6 columns]


- 해당 값(-33.21928095)을 제외하면,`target` 변수의 값의 범위는 -17 ~ 18이다.

- 종합해 보자면, 아래의 세 가지 이유로 해당 값(-33.21928095)을 결측치로 볼 수 있다.
    1. 소수점 8자리까지 정확하게 일치하는 값이 `feature_1, 2, 3` 세 변수의 모든 카테고리 레벨에서 등장한다.
    2. 다른 값들과 비교했을 때, 해당 값의 개수가 지나치게 많다.
    3. 해당 값을 제외한 다른 값들의 범위(-17 ~ 18)와 지나치게 차이가 많이 난다.
- 또한, 경험적인 이유에서도 해당 값이 결측치로 의심스럽다.
    1. 현업에선 결측치를 NaN으로 넣는 것보다 미리 협의된 특정값으로 넣는 게 편하기 때문에 이러한 방식을 많이 쓴다고 알고 있다.
    2. Kaggle의 Competetion이었던 KKBox's Music Recommendation Challenge에서도 사용자가 입력하지 않고 Skip한 값을 결측치가 아닌 -1로 표시했다.
- 해당 변수(`target`)가 충성도를 의미한다는 것을 생각한다면, 아마 충성도를 계산할 수 없을 정도로 정보가 적은 고객들은 충성도 값을 결측치 대신 해당 값(-33.21928095)으로 넣은 것이라 추측했다. 이후의 EDA를 통해 이를 확인해 볼 예정이다.

- 해당 값을 결측치로 볼 수 있는지 판단을 위해 아래의 검증 과정을 거침
    - 만약 충성도를 계산할 수 없을 정도의 고객이라면 거래 빈도가 낮을 것이다. 즉, transactions 건수 자체가 적을 것이다.
        - 해당 값을 가지는 `card_id`와 해당 값이 아닌 나머지의 `card_id`의 빈도 평균 또는 전체 `card_id`의 빈도 평균과 비교해보자.

# 3. historical_transactions와 merge

In [14]:
trans = pd.read_csv('../input/historical_transactions.csv')

In [15]:
trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [16]:
# `trans`에서 `card_id`들의 row를 count
trans_card_count = pd.DataFrame(trans['card_id'].value_counts().reset_index().rename(columns={'card_id':'count', 'index':'card_id'}))
trans_card_count.head()

,card_id,count
0,C_ID_3d3dfdc692,5582
1,C_ID_0cd2ce025c,2912
2,C_ID_cc3d4cd4e3,2143
3,C_ID_5ccc07beb9,2066
4,C_ID_5ea401d358,1786


In [17]:
train_copy = train[['card_id', 'target']]
train_copy.head()

,card_id,target
0,C_ID_92a2005557,-0.820283
1,C_ID_3d0044924f,0.392913
2,C_ID_d639edf6cd,0.688056
3,C_ID_186d6a6901,0.142495
4,C_ID_cdbd2c0db2,-0.159749


In [18]:
# 두 테이블 병합
merged = train_copy.merge(trans_card_count, how='inner', on='card_id')
merged.head()

,card_id,target,count
0,C_ID_92a2005557,-0.820283,260
1,C_ID_3d0044924f,0.392913,350
2,C_ID_d639edf6cd,0.688056,43
3,C_ID_186d6a6901,0.142495,77
4,C_ID_cdbd2c0db2,-0.159749,133


In [19]:
# target이 특정값인 경우에만 null 컬럼의 값에 1을 부여
merged['null'] = [1 if x == merged['target'].min() else 0 for x in merged['target']]
merged.head()

,card_id,target,count,null
0,C_ID_92a2005557,-0.820283,260,0
1,C_ID_3d0044924f,0.392913,350,0
2,C_ID_d639edf6cd,0.688056,43,0
3,C_ID_186d6a6901,0.142495,77,0
4,C_ID_cdbd2c0db2,-0.159749,133,0


In [20]:
# 여기서 말하는 평균 빈도는 해당 card_id가 historical_transactions의 row count를 의미한다.
print('1. 전체 card_id의 평균 빈도: {}'.format(round(merged['count'].mean(),2)))
print('2. 특정값을 충성도로 갖는 card_id의 평균 빈도: {}'.format(round(merged[merged['null'] == 1]['count'].mean(),2)))
print('3. 특정값을 충성도로 갖는 card_id를 제외한 전체의 평균 빈도: {}'.format(round(merged[merged['null'] == 0]['count'].mean(),2)))

1. 전체 card_id의 평균 빈도: 89.29
2. 특정값을 충성도로 갖는 card_id의 평균 빈도: 81.61
3. 특정값을 충성도로 갖는 card_id를 제외한 전체의 평균 빈도: 89.38


- 전체 `card_id`의 평균 빈도와 특정값을 충성도로 갖는 `card_id`의 평균 빈도, 특정값을 충성도로 갖는 `card_id`를 제외한 전체의 평균 빈도 모두 큰 차이가 없었다.
- 특정값을 충성도로 갖는 `card_id`의 평균 빈도는 나머지 두 값보다 약 8정도 낮았지만, 결측치로 판단해 특정값을 넣을 만큼 차이가 크거나 값이 낮다고 보기 힘들다.
- 따라서 이전에 설정한 가설이었던 '거래 내역의 빈도가 적어 특정 `card_id`는 충성도를 특정값으로 넣었을 것이다'는 기각할 수 있다.